In [23]:
import pandas as pd
from nltk import tokenize
df = pd.read_csv('sample_text.csv')

In [24]:
ls_text = []

for i in range(len(df)):
    ls_text.append(tokenize.sent_tokenize(df['context'][i]))

# Sentence Transformers method 1

In [25]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')

ans_1 = []
scores = []
for k in range(len(df)):
    print('\n\n')
    #Compute embedding for both lists
    embeddings1 = model.encode(ls_text[k], convert_to_tensor=True)

    a = np.matmul(embeddings1,embeddings1.T)
    a = np.abs(a).numpy()
    a = np.sum(a, axis = 1)
    temp = []
    temp2 = []
    for i in np.argpartition(a, -10)[-10:]:
        # print(ls_text[k][i])
        temp.append(ls_text[k][i])
        print(ls_text[k][i])
        temp2.append(a[i])

        
    # for i in np.argpartition(a, -10)[-10:]:
    #     # print(ls_text[k][i])
    #     temp.append(ls_text[k][i])
    #     # print(ls_text[k][i])
    #     print(a[i])

    ans_1.append(temp)
    scores.append(temp2)


    #Compute cosine-similarities
    # cosine_scores = util.cos_sim(embeddings1, embeddings2)

    # #Output the pairs with their score
    # for i in range(len(sentences1)):
    #     print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))






This type of symbiosis is relatively uncommon in rudimentary reference texts, but is omnipresent in the natural world.
An example of mutual symbiosis is the relationship between the ocellaris clownfish that dwell among the tentacles of Ritteri sea anemones.
Another non-obligate symbiosis is known from encrusting bryozoans and hermit crabs that live in a close relationship.
Symbiotic relationships include those associations in which one organism lives on another (ectosymbiosis, such as mistletoe), or where one partner lives inside the other (endosymbiosis, such as lactobacilli and other bacteria in humans or Symbiodinium in corals).
While historically, symbiosis has received less attention than other interactions such as predation or competition, it is increasingly recognized as an important selective force behind evolution, with many species having a long history of interdependent co-evolution.
Many biologists restrict the definition of symbiosis to close mutualist relationships.
An

In [26]:
df1 = pd.DataFrame(ans_1).T
df1.to_csv('sent_trans_1.csv')
df1 = pd.DataFrame(scores).T
df1.to_csv('sent_trans_1_scores.csv')


# Sentence Transformers method 2

In [59]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

ans_2 = []
scores = []
for i in range(len(df)):
    sents = ls_text[i]
    
    # Query sentences:
    queries = [df['title'][i]]
    corpus_embeddings = embedder.encode(sents, convert_to_tensor=True)



    # Find the closest 10 sentences of the corpus for each query sentence based on cosine similarity

    top_k = min(10, len(sents))
    for query in queries:
        query_embedding = embedder.encode(query, convert_to_tensor=True)

        # We use cosine-similarity and torch.topk to find the highest 1- scores
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)

        print("\n\n======================\n\n")
        print("Query:", query)
        print("\nTop 10 most similar sentences in corpus:")
        
        temp = []
        temp2 = []
        for score, idx in zip(top_results[0], top_results[1]):
            # print(sents[idx], "(Score: {:.4f})".format(score))
            temp.append(sents[idx])
            temp2.append(float("{:.4f}".format(score)))
        ans_2.append(temp)
        scores.append(temp2)
        





Query: Symbiosis

Top 10 most similar sentences in corpus:




Query: Samoa

Top 10 most similar sentences in corpus:




Query: House_music

Top 10 most similar sentences in corpus:




Query: Czech_language

Top 10 most similar sentences in corpus:




Query: Dialect

Top 10 most similar sentences in corpus:




Query: Hanover

Top 10 most similar sentences in corpus:




Query: Digestion

Top 10 most similar sentences in corpus:




Query: IBM

Top 10 most similar sentences in corpus:




Query: Neolithic

Top 10 most similar sentences in corpus:




Query: Sexual_orientation

Top 10 most similar sentences in corpus:


In [60]:
scores[0]

[0.7749, 0.7628, 0.7476, 0.6967, 0.6674, 0.6501, 0.646, 0.6346, 0.6177, 0.6036]

In [63]:
df2 = pd.DataFrame(ans_2).T
# df2.to_csv('sent_trans_2.csv')
df2 = pd.DataFrame(scores).T
df2.to_csv('sent_trans_2_scores.csv')

# TF-IDF method

In [29]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    lemma = WordNetLemmatizer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = lemma.lemmatize(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [30]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [31]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [32]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [33]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [34]:
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [35]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [38]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = []
    sco = []

    ls = np.argpartition(list(sentenceValue.values()), -10)[-10:]
    threshold = list(sentenceValue.values())[ls[0]]
    # ans = []
    # for i in ls:
    #     ans.append(sentences)
    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary.append(sentence)
            sentence_count += 1
            sco.append(sentenceValue[sentence[:15]])

    return summary,sco

In [39]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mihir\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [41]:
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords    
    
'''
We already have a sentence tokenizer, so we just need 
to run the sent_tokenize() method to create the array of sentences.
'''

ans_3 = []
scores = []

for i in range(len(ls_text)):

    sentences = ls_text[i]
    # 1 Sentence Tokenize
    # sentences = sent_tokenize(text)
    total_documents = len(sentences)
    #print(sentences)

    # 2 Create the Frequency matrix of the words in each sentence.
    freq_matrix = _create_frequency_matrix(sentences)
    #print(freq_matrix)

    '''
    Term frequency (TF) is how often a word appears in a document, divided by how many words are there in a document.
    '''
    # 3 Calculate TermFrequency and generate a matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    '''
    Inverse document frequency (IDF) is how unique or rare a word is.
    '''
    # 5 Calculate IDF and generate a matrix
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(tf_idf_matrix)
    #print(sentence_scores)

    # 8 Find the threshold
    threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary
    summary,sco = _generate_summary(sentences, sentence_scores, threshold)
    # print(summary)
    ans_3.append(summary)
    scores.append(sco)
    

In [42]:
df2 = pd.DataFrame(ans_3).T
df2.to_csv('tf-idf.csv')
df2 = pd.DataFrame(scores).T
df2.to_csv('tf-idf_scores.csv')

In [51]:
df['sentence_transformer_method1'] = ans_1
df['sentence_transformer_method2'] = ans_2
df['tf_idf'] = ans_3
df.to_csv('sample_text_results_1.csv', index=False)

In [1]:
from summa import keywords

In [2]:
summary = """Neural networks, also known as artificial neural networks (ANNs) or simulated neural networks (SNNs), are a subset of machine learning and are at the heart of deep learning algorithms. Their name and structure are inspired by the human brain, mimicking the way that biological neurons signal to one another.

Artificial neural networks (ANNs) are comprised of a node layers, containing an input layer, one or more hidden layers, and an output layer. Each node, or artificial neuron, connects to another and has an associated weight and threshold. If the output of any individual node is above the specified threshold value, that node is activated, sending data to the next layer of the network. Otherwise, no data is passed along to the next layer of the network.

Neural networks rely on training data to learn and improve their accuracy over time. However, once these learning algorithms are fine-tuned for accuracy, they are powerful tools in computer science and artificial intelligence, allowing us to classify and cluster data at a high velocity. Tasks in speech recognition or image recognition can take minutes versus hours when compared to the manual identification by human experts. One of the most well-known neural networks is Google’s search algorithm.

Once an input layer is determined, weights are assigned. These weights help determine the importance of any given variable, with larger ones contributing more significantly to the output compared to other inputs. All inputs are then multiplied by their respective weights and then summed. Afterward, the output is passed through an activation function, which determines the output. If that output exceeds a given threshold, it “fires” (or activates) the node, passing data to the next layer in the network. This results in the output of one node becoming in the input of the next node. This process of passing data from one layer to the next layer defines this neural network as a feedforward network.

If we use the activation function from the beginning of this section, we can determine that the output of this node would be 1, since 6 is greater than 0. In this instance, you would go surfing; but if we adjust the weights or the threshold, we can achieve different outcomes from the model. When we observe one decision, like in the above example, we can see how a neural network could make increasingly complex decisions depending on the output of previous decisions or layers.

In the example above, we used perceptrons to illustrate some of the mathematics at play here, but neural networks leverage sigmoid neurons, which are distinguished by having values between 0 and 1. Since neural networks behave similarly to decision trees, cascading data from one node to another, having x values between 0 and 1 will reduce the impact of any given change of a single variable on the output of any given node, and subsequently, the output of the neural network.

Neural networks can be classified into different types, which are used for different purposes. While this isn’t a comprehensive list of types, the below would be representative of the most common types of neural networks that you’ll come across for its common use cases:

The perceptron is the oldest neural network, created by Frank Rosenblatt in 1958.

Feedforward neural networks, or multi-layer perceptrons (MLPs), are what we’ve primarily been focusing on within this article. They are comprised of an input layer, a hidden layer or layers, and an output layer. While these neural networks are also commonly referred to as MLPs, it’s important to note that they are actually comprised of sigmoid neurons, not perceptrons, as most real-world problems are nonlinear. Data usually is fed into these models to train them, and they are the foundation for computer vision, natural language processing, and other neural networks.

Convolutional neural networks (CNNs) are similar to feedforward networks, but they’re usually utilized for image recognition, pattern recognition, and/or computer vision. These networks harness principles from linear algebra, particularly matrix multiplication, to identify patterns within an image.

Recurrent neural networks (RNNs) are identified by their feedback loops. These learning algorithms are primarily leveraged when using time-series data to make predictions about future outcomes, such as stock market predictions or sales forecasting.

Deep Learning and neural networks tend to be used interchangeably in conversation, which can be confusing. As a result, it’s worth noting that the “deep” in deep learning is just referring to the depth of layers in a neural network. A neural network that consists of more than three layers—which would be inclusive of the inputs and the output—can be considered a deep learning algorithm. A neural network that only has two or three layers is just a basic neural network.

"""

In [3]:
TR_keywords = keywords.keywords(summary, scores=True)
print(TR_keywords[0:10])

[('networks', 0.37653801069261195), ('network', 0.37653801069261195), ('layers', 0.2578096552459131), ('layer', 0.2578096552459131), ('neural', 0.25425390792625246), ('data', 0.247348041933658), ('decision', 0.16509210413479947), ('decisions', 0.16509210413479947), ('neurons', 0.15753549557120922), ('neuron', 0.15753549557120922)]


In [6]:
!pip install keybert

  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Created wheel for keybert: filename=keybert-0.5.1-py3-none-any.whl size=21332 sha256=7f4da0a0827a80cbf122ed41c197f6d28d1deb5a869f7f4cff100b95bf72ddd3
  Stored in directory: c:\users\mihir\appdata\local\pip\cache\wheels\94\18\2a\f26bbcd25924aab452bb4bcc2345a55c07160823d196a264c7
Successfully built keybert


In [4]:
from keybert import KeyBERT

In [5]:
kw_model = KeyBERT(model='all-mpnet-base-v2')

In [6]:
keywords = kw_model.extract_keywords(summary, 

                                     keyphrase_ngram_range=(1, 3), 

                                     stop_words='english', 

                                     highlight=False,

                                     top_n=10)

keywords_list= list(dict(keywords).keys())

print(keywords_list)

['neural networks google', 'neural network', 'processing neural networks', 'neural networks snns', 'neural networks anns', 'layer network neural', 'network neural network', 'network neural', 'algorithm neural', 'neural network layers']


# cdj

In [8]:

#### Pytextrank to find important phrases

import spacy
import pytextrank

# example text
# text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."
text = summary
# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_lg")

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")
doc = nlp(text)

# examine the top-ranked phrases in the document
for phrase in doc._.phrases:
    print(phrase.text)
    print(phrase.rank, phrase.count)
    print(phrase.chunks)

Neural networks
0.12890204814099 3
[Neural networks, Neural networks, Neural networks]
neural networks
0.12890204814099 4
[neural networks, neural networks, neural networks, neural networks]
other neural networks
0.12553041726409025 1
[other neural networks]
Artificial neural networks
0.12399304051033513 1
[Artificial neural networks]
artificial neural networks
0.12399304051033513 1
[artificial neural networks]
simulated neural networks
0.1204046985684943 1
[simulated neural networks]
Convolutional neural networks
0.11919226027186872 1
[Convolutional neural networks]
Feedforward neural networks
0.11919226027186872 1
[Feedforward neural networks]
Recurrent neural networks
0.11919226027186872 1
[Recurrent neural networks]
feedforward networks
0.10376433508421891 1
[feedforward networks]
layers
0.09455237544804192 3
[layers, layers, layers]
deep learning algorithms
0.07252282768046249 1
[deep learning algorithms]
the oldest neural network
0.0714232241661447 1
[the oldest neural network]
o